In [1]:
import numpy as np
import pandas as pd

# viz
import matplotlib.pyplot as plt

# notebook settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 1000)

## Sample Prep

In [2]:
samples = pd.read_csv('../data/TCGA/gdc_sample_sheet.2019-12-09.tsv', sep="\t")
# get file type
samples['data'] = [val[1] for i,val in samples['File Name'].str.split(".").items()]
samples.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,data
0,c5b283b8-a6ab-4652-b824-18fe1cebe0e3,0bf53fc6-b8fb-4e6d-9297-4129c708f3da.FPKM.txt.gz,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUAD,TCGA-50-5935,TCGA-50-5935-01A,Primary Tumor,FPKM
1,27079945-f3da-42b4-8c17-c82d66b7d321,f4a1e772-2a5e-4c0a-803a-1bdb2b376a47.FPKM-UQ.t...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUAD,TCGA-55-6971,TCGA-55-6971-11A,Solid Tissue Normal,FPKM-UQ
2,da368838-dad0-434d-a2a8-084b362e358e,12d2b9fa-1921-4033-bdb9-7e114c0d7812.FPKM.txt.gz,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUAD,TCGA-99-8025,TCGA-99-8025-01A,Primary Tumor,FPKM
3,be46ee72-8958-483f-bd54-38d21ebf7ff6,4d36e127-5fad-4b97-afff-28f4bdbf5f5d.FPKM.txt.gz,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUAD,TCGA-55-8301,TCGA-55-8301-01A,Primary Tumor,FPKM
4,c5f764bd-a9d4-4188-a4dc-a5e6ca1183f8,c57c5f57-5426-4662-832d-6f342ebeff04.FPKM-UQ.t...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUAD,TCGA-73-4677,TCGA-73-4677-01A,Primary Tumor,FPKM-UQ


Samples with RNAseq adjacent normal tissue

In [3]:
samples[samples['Sample Type']=='Solid Tissue Normal']['data'].value_counts()

FPKM       59
FPKM-UQ    59
htseq      59
Name: data, dtype: int64

In [4]:
# all cases with adjacent normal tissue
cases = samples[samples['Sample Type']=='Solid Tissue Normal']['Case ID']

In [5]:
# disparity in cases
samples[(samples['Case ID'].isin(cases)) & (samples['Sample Type']=='Primary Tumor') & (samples['data']=='FPKM')]['Case ID'].nunique()
samples[(samples['Case ID'].isin(cases)) & (samples['Sample Type']=='Solid Tissue Normal') & (samples['data']=='FPKM')]['Case ID'].nunique()

57

59

In [6]:
# divide, join, subset
case_tumor = samples[(samples['Case ID'].isin(cases)) & (samples['Sample Type']=='Primary Tumor') & (samples['data']=='FPKM')]
case_norm = samples[(samples['Case ID'].isin(cases)) & (samples['Sample Type']=='Solid Tissue Normal') & (samples['data']=='FPKM')]
cases = case_norm[case_norm['Case ID'].isin(case_tumor['Case ID'])]['Case ID']
cases.shape

(57,)

In [7]:
case_tumor = case_tumor[case_tumor['Case ID'].isin(cases)]
case_norm = case_norm[case_norm['Case ID'].isin(cases)]
cases = pd.concat([case_tumor, case_norm])
cases['Sample Type'] = cases['Sample Type'].astype('category')

In [8]:
case_tumor.shape
case_norm.shape
cases.shape

(67, 9)

(57, 9)

(124, 9)

## Dataset Prep

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cases)
train, test = cases.copy(), cases.sample(n=20).copy()
train['Sample Type'].value_counts()
test['Sample Type'].value_counts()

Primary Tumor          67
Solid Tissue Normal    57
Name: Sample Type, dtype: int64

Primary Tumor          11
Solid Tissue Normal     9
Name: Sample Type, dtype: int64

In [10]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit
import numpy as np
cuda = torch.cuda.is_available()
print("Cuda is available: {}".format(cuda))

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']
classes = train['Sample Type'].cat.categories.values

def plot_embeddings(embeddings, targets, xlim=None, ylim=None):
    plt.figure(figsize=(10,10))
    for i in range(10):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(classes)

def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader), 2))
        labels = np.zeros(len(dataloader))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

Cuda is available: True


In [ ]:
from tcga_datasets import TCGA, SiameseTCGA
root_dir = "../data/TCGA"
batch_size = 1

train_dataset = TCGA(root_dir, samples=train, train=True)
test_dataset = TCGA(root_dir, samples=test, train=False)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)


## Siamese Network

In [ ]:
# Step 1 set up dataloader
root_dir = "../data/TCGA"
siamese_train_dataset = SiameseTCGA(train_dataset) # Returns pairs of images and target same/different
siamese_test_dataset = SiameseTCGA(test_dataset)
batch_size = 10
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
siamese_train_loader = torch.utils.data.DataLoader(siamese_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
siamese_test_loader = torch.utils.data.DataLoader(siamese_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from tcga_networks import EmbeddingNet, SiameseNet
from losses import ContrastiveLoss
from metrics import AccumulatedAccuracyMetric

# Step 2
embedding_net = EmbeddingNet()
# Step 3
model = SiameseNet(embedding_net)
if cuda:
    model.cuda()
    
# Step 4
margin = 1.
loss_fn = ContrastiveLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
# print training metrics every log_interval * batch_size
log_interval = 3

In [ ]:
train_loss, val_loss = fit(siamese_train_loader, siamese_test_loader, model, loss_fn, optimizer, scheduler, 
    n_epochs, cuda, log_interval)

In [ ]:
plt.plot(range(0, n_epochs), train_loss, 'rx-')
plt.plot(range(0, n_epochs), val_loss, 'bx-')

In [ ]:
train_embeddings_cl, train_labels_cl = extract_embeddings(train_loader, model)
plot_embeddings(train_embeddings_cl, train_labels_cl)

In [ ]:
val_embeddings_baseline, val_labels_baseline = extract_embeddings(test_loader, model)
plot_embeddings(val_embeddings_baseline, val_labels_baseline)

In [ ]:
len(train_dataset.labels)

In [ ]:
from datasets import BalancedBatchSampler

train_batch_sampler = BalancedBatchSampler(train_dataset.labels, n_classes=2, n_samples=60)
test_batch_sampler = BalancedBatchSampler(test_dataset.labels, n_classes=2, n_samples=25)

In [36]:
for label in train_batch_sampler:
    len(label)
    break

117

In [52]:
siamese_train_dataset.train_data

tensor([[0.0000e+00, 0.0000e+00, 4.9478e+00,  ..., 0.0000e+00, 4.2358e+00,
         0.0000e+00],
        [0.0000e+00, 6.3250e-03, 1.2249e+00,  ..., 0.0000e+00, 7.3963e-01,
         0.0000e+00],
        [2.6798e-01, 2.9746e-01, 2.3197e+00,  ..., 0.0000e+00, 2.1039e+00,
         2.3825e-01],
        ...,
        [0.0000e+00, 0.0000e+00, 3.4065e+00,  ..., 0.0000e+00, 8.1920e-01,
         0.0000e+00],
        [7.5395e-02, 2.6153e-03, 3.3482e+00,  ..., 0.0000e+00, 9.4216e-01,
         0.0000e+00],
        [6.7487e-02, 0.0000e+00, 3.1826e+00,  ..., 0.0000e+00, 7.6828e-01,
         0.0000e+00]])

In [21]:
siamese_train_dataset.labels_set

{0, 1}

In [23]:
foo = {label: np.where(siamese_train_dataset.train_labels == label)[0]
                                     for label in siamese_train_dataset.labels_set}
foo

{0: array([ 0,  2,  3,  5,  7,  9, 10, 11, 12, 13, 15, 18, 19, 20, 21, 23, 25,
        26, 27, 28, 29, 33, 34, 40, 42, 45, 47, 49, 52, 55, 59, 60, 61, 65,
        66, 68, 71, 73, 74, 76, 78, 79, 83, 86, 87, 88, 89, 90, 91, 92]),
 1: array([ 1,  4,  6,  8, 14, 16, 17, 22, 24, 30, 31, 32, 35, 36, 37, 38, 39,
        41, 43, 44, 46, 48, 50, 51, 53, 54, 56, 57, 58, 62, 63, 64, 67, 69,
        70, 72, 75, 77, 80, 81, 82, 84, 85])}

In [ ]:
[[i, random_state.choice(self.label_to_indices[self.test_labels[i].item()]), 1] for i in range(0, len(self.test_data), 2)]


In [27]:
len(siamese_test_dataset.test_data)
siamese_test_dataset.label_to_indices
siamese_test_dataset.test_labels

31

{0: array([ 1,  4,  5,  6,  7, 10, 11, 13, 14, 15, 17, 18, 22, 23, 24, 28, 30]),
 1: array([ 0,  2,  3,  8,  9, 12, 16, 19, 20, 21, 25, 26, 27, 29])}

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0])

In [24]:
siamese_test_dataset.test_pairs

[[0, 12, 1],
 [2, 8, 1],
 [4, 22, 1],
 [6, 5, 1],
 [8, 20, 1],
 [10, 1, 1],
 [12, 21, 1],
 [14, 4, 1],
 [16, 20, 1],
 [18, 18, 1],
 [20, 26, 1],
 [22, 10, 1],
 [24, 24, 1],
 [26, 19, 1],
 [28, 1, 1],
 [30, 7, 1],
 [1, 3, 0],
 [3, 28, 0],
 [5, 16, 0],
 [7, 19, 0],
 [9, 6, 0],
 [11, 0, 0],
 [13, 25, 0],
 [15, 26, 0],
 [17, 20, 0],
 [19, 6, 0],
 [21, 24, 0],
 [23, 27, 0],
 [25, 24, 0],
 [27, 23, 0],
 [29, 23, 0]]